In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import networkx as nx

df = pd.read_csv('movie_industry.csv', encoding="windows-1252")
df2 = pd.read_csv('the_oscar_award.csv')
genome_scores = pd.read_csv('genome-scores.csv')
genome_tags = pd.read_csv('genome-tags.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')
movies = pd.read_csv('movies.csv')

class Graph:
    def __init__(self, 
                 directors_to_actors_relation, 
                 weight_func, 
                 weight_func_args, 
                 director_column="director", 
                 actor_column="star", 
                 bipartite=True):
        self.G = nx.Graph()
        
        directors = set(directors_to_actors_relation[director_column].values)
        actors = set(directors_to_actors_relation[actor_column].values)
        
        for director in directors:
            self.G.add_node((director, True))
        
        for actor in actors:
            self.G.add_node((actor, False))

        for director in directors:
            rows = directors_to_actors_relation[directors_to_actors_relation[director_column] == director]
            for index in rows.index.values:
                self.G.add_edge((director, True), (rows.loc[index, actor_column], False), 
                                weight=weight_func(*[rows.loc[index, i] for i in weight_func_args]))
                

def example_weight_func(director, star):
    return len(df.loc[((df["director"] == director) & (df["star"] == "Clint Eastwood"))])

storage = Graph(df, example_weight_func, ["director", "star"])